In [1]:
import math
import ipywidgets as widgets
from ipywidgets import interactive, IntProgress, TwoByTwoLayout
from latex_results import load_measure_for_classifiers_pickles, normalized_absolute_error, LR_KEY, MNB_KEY, SVM_KEY, RF_KEY
from plotly_colors import colors
import random
import numpy as np
from IPython.display import display
from load_data import get_measures_mean_across_experiments, jupyter_measures
from drift_analysis import extract_binary_data
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def extract_measures(meas_mean, isomerous):
    nae = np.array(
        [normalized_absolute_error(em_pr, test_pr) for em_pr, test_pr in zip(meas_mean.em_priors[1], meas_mean.test_priors[1])]
    )
    if isomerous:
        cal = np.array(meas_mean.isomerous_em_cal_loss[1])
        ref = np.array(meas_mean.isomerous_em_ref_loss[1])
    else:
        cal = np.array(meas_mean.isometric_em_cal_loss[1])
        ref = np.array(meas_mean.isometric_em_ref_loss[1])

    brier = cal + ref
    return nae, brier, cal, ref

MINORITY_CLASS, RANDOM_CLASS = range(2)

<a id='plot_mean'></a>
## Visualize NAE, BS, CE and RE plots
In this section you can select the experiments on SLD based on the number of target classes (2, 5, 10, 20, 37) and
visualize the plots for each of the above measures. The data is averaged on the 500 experiments run as indicated
in *Paper ref*

In [3]:

def plot_experiments_for_class(class_):
    progress = IntProgress(min=0, max=100, description="Loading data...", bar_style='info', style={'description_width': 'initial'})
    display(progress)
    # Sort by classifier name
    measures = sorted(load_measure_for_classifiers_pickles("rcv1", class_), key=lambda m: m[0])
    global fig
    if fig is not None and type(fig) is go.FigureWidget:
        fig.close()

    if class_ == "2":
        measures = list(map(lambda m: (m[0], list(m[1])), extract_binary_data(measures)))

    clf_data = {j: {} for j in jupyter_measures.keys()}
    max_len = 0
    progress.description = 'Extracting data...'
    progress.value = 0
    for clf_name, data in measures:
        measure_mean = get_measures_mean_across_experiments(data)
        nae, brier, cal, ref = extract_measures(measure_mean, True)
        if (l := nae.shape[0]) > max_len:
            max_len = l
        clf_data['nae'][clf_name] = nae
        clf_data['bs'][clf_name] = brier
        clf_data['ce'][clf_name] = cal
        clf_data['re'][clf_name] = ref
        progress.value += 100 / len(measures)

    fig = make_subplots(rows=2, cols=2, subplot_titles=[v for v in sorted(jupyter_measures.values())])
    i = 0
    progress.description = 'Plotting data...'
    progress.value = 0
    for metric, clf_measures in sorted(clf_data.items(), key=lambda kv: kv[0]):
        picked_colors = set()
        for clf_name, value in clf_measures.items():
            row = math.ceil((i+1) / 2)
            col = 1 if (i+1) % 2 == 0 else 2
            pad_width = (0, max_len - len(value))
            if len(value.shape) > 1:
                padded = np.pad(value, (pad_width, (0, 0)), mode='edge')
            else:
                padded = np.pad(value, pad_width, mode='edge')

            random.seed(a=hash(clf_name))
            color = random.choice(colors)

            # Keep selecting a color until we find one we haven't already used
            while color in picked_colors:
                color = random.choice(colors)
            picked_colors.add(color)
            fig.add_trace(
                go.Scatter(
                    x=np.array(range(max_len)),
                    y=padded,
                    mode='lines',
                    name=clf_name,
                    visible='legendonly',
                    legendgroup=clf_name,
                    showlegend=row == 1 and col == 1,
                    line={'color': color}
                ), row=row, col=col
            )
        i += 1
        progress.value += 100 / len(clf_data)

    fig.update_layout(autosize=False, width=900, height=400 * (math.log2(len(jupyter_measures))))
    fig.update_xaxes(automargin=True, title_text="SLD iterations")
    fig.update_yaxes(automargin=True)
    progress.close()
    fig = go.FigureWidget(fig)
    display(fig)

                
fig = None

w = widgets.Dropdown(options=['2', '5', '10', '20', '37'], value='5', description='Target classes', disabled=False,
                   style={'description_width': 'initial'})
w_disp = interactive(plot_experiments_for_class, class_=w)
display(w_disp)

interactive(children=(SelectMultiple(description='Metrics', options=(('Abs. err.', 'abs_errors'), ('Brier', 'b…

## Compare metrics on two random samples
In this section you can compare several metrics on two randomly chosen samples. You can select one out of the seven
available classifiers we used to run our experiments.

In [ ]:
def plot_two_random_sample(_):
    progress = IntProgress(min=0, max=100, description="Loading data...", bar_style='info', style={'description_width': 'initial'})
    display(progress)
    measures = list(filter(lambda k: k[0] == select_classifier.value, load_measure_for_classifiers_pickles("rcv1", select_class.value)))
    global fig, fig2
    if fig is not None and type(fig) is go.FigureWidget:
        fig.close()
    if fig2 is not None and type(fig2) is go.FigureWidget:
        fig2.close()

    progress.value = 50
    if select_class.value == "2":
        measures = list(map(lambda m: (m[0], list(m[1])), extract_binary_data(measures)))

    random_sample = random.choice(measures[0][1])
    max_len = len(random_sample.em_priors[1])
    fig = go.FigureWidget()

    if select_target.value == MINORITY_CLASS:
        target_class = np.argmin(random_sample.train_priors[1][0]) if hasattr(random_sample.train_priors[1][0], '__len__') else 0
    else:
        target_class = random.choice(range(int(select_class.value))) if select_class.value != "2" else 0

    test_priors = random_sample.test_priors[1][0]
    test_priors = np.array([test_priors[0][target_class]]) if len(test_priors.shape) > 1 else np.array([test_priors])
    em_priors = np.array(random_sample.em_priors[1])
    train_priors = random_sample.train_priors[1][0]
    train_priors = np.array([train_priors[0][target_class]]) if len(train_priors.shape) > 1 else np.array([train_priors])
    # At iteration 0 we show the true train priors (which are our initial priors in the SLD
    # algorithm, see em.py and em_test.py in this repository)
    em_priors[0] = train_priors
    nae, brier, cal, ref = extract_measures(random_sample, True)
    progress.description = 'Plotting data...'

    for name, value in [('Prior in unlabelled set', test_priors),('Prior in labelled set', train_priors),
                        ('Prior generated by SLD', em_priors)]:
        pad_width = (0, max_len - len(value))
        if len(value.shape) > 1:
            padded = np.pad(value, (pad_width, (0, 0)), mode='edge')
        else:
            padded = np.pad(value, pad_width, mode='edge')

        fig.add_trace(
            go.Scatter(
                x=np.array(range(max_len)),
                y=padded[:, target_class] if len(value.shape) > 1 else padded,
                mode='lines',
                name=name,
                visible='legendonly',
                legendgroup=name,
                # line={'color': color},
            )
        )
        progress.value += 7

    fig2 = go.FigureWidget()
    for name, value in [('NAE', nae), ('BS', brier), ('CE', cal), ('RE', ref)]:
        pad_width = (0, max_len - len(value))
        if len(value.shape) > 1:
            padded = np.pad(value, (pad_width, (0, 0)), mode='edge')
        else:
            padded = np.pad(value, pad_width, mode='edge')

        fig2.add_trace(
            go.Scatter(
                x=np.array(range(max_len)),
                y=padded[:, target_class] if len(value.shape) > 1 else padded,
                mode='lines',
                name=name,
                visible='legendonly',
                legendgroup=name,
                # line={'color': color},
            )
        )
        progress.value += 7

    fig.update_layout(xaxis_title='SLD Iterations', title='Priors', legend=dict(y=1.2, x=.2, orientation='h'))
    fig2.update_layout(xaxis_title='SLD Iterations', title='Metrics', legend=dict(y=1.2, x=.3, orientation='h'))
    progress.close()
    display(fig)
    display(fig2)


classifier_keys = [LR_KEY, RF_KEY, MNB_KEY]
classifier_keys = sorted(["Calibrated-" + c for c in classifier_keys] + classifier_keys + [SVM_KEY])
classifier_options = [(c.replace('-', ' '), c) for c in classifier_keys]
fig = None
fig2 = None

select_classifier = widgets.Dropdown(options=classifier_options, description="Classifier")
select_class = widgets.Dropdown(options=['2', '5', '10', '20', '37'], value='5', description='Target classes',
                                style={'description_width': 'initial'})
select_target = widgets.Dropdown(options=[("Minority class", MINORITY_CLASS), ("Random class", RANDOM_CLASS)],
                                 description='Plot class')
confirm_button = widgets.Button(description='Confirm', button_style='info')
confirm_button.on_click(plot_two_random_sample)

TwoByTwoLayout(
    top_left=select_classifier,
    top_right=select_target,
    bottom_left=select_class,
    bottom_right=confirm_button,
)
